<a href="https://colab.research.google.com/github/PetrongariYanina/AirBnb/blob/main/Mini_project_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importación del Dataset

In [13]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd


In [14]:
df= pd.read_csv('/cleaned_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4729 entries, 0 to 4728
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   clean_tweet  4729 non-null   object
 1   Ofensivo     4729 non-null   int64 
 2   Vulgar       4729 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 111.0+ KB


In [15]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


tweets = df['clean_tweet'].tolist()


class TweetDataset(Dataset):
    def init(self, tweets):
        self.tweets = tweets

    def len(self):
        return len(self.tweets)

    def getitem(self, idx):
        return self.tweets[idx]

df['Vulgar'] = df['Vulgar'].astype('bool')
df['Ofensivo'] = df['Ofensivo'].astype('bool')


## Carga del Modelo

In [17]:
pip install datasets

In [18]:
pip install datasets

In [25]:
print(examples)

NameError: name 'examples' is not defined

In [19]:

import torch
from torch.utils.data import DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
from tqdm import tqdm

# ======== 1. Preparación del Dataset ========
# Dividir los datos en conjunto de entrenamiento y validación
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['clean_tweet'],  # Textos
    df[['Vulgar', 'Ofensivo']].values,  # Etiquetas multietiqueta como matriz numpy
    test_size=0.2,
    random_state=42
)

# Convertir los datos a Dataset de Hugging Face
train_data = Dataset.from_dict({"clean_tweet": train_texts, "labels": train_labels.tolist()})
val_data = Dataset.from_dict({"clean_tweet": val_texts, "labels": val_labels.tolist()})

# ======== 2. Tokenización ========
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["clean_tweet"], padding="max_length", truncation=True, max_length=128)

# Tokenizar los datasets
train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)

# Formato PyTorch
train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# ======== 3. Crear DataLoaders ========
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8)

# ======== 4. Modelo ========
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2  # Dos etiquetas: Vulgar y Ofensivo
)

# ======== 5. Configuración del Optimizer y Scheduler ========
# Primera fase: Congelar las primeras 4 capas
optimizer = AdamW([
    {"params": model.distilbert.transformer.layer[:4].parameters(), "lr": 1e-5},
    {"params": model.distilbert.transformer.layer[4:].parameters(), "lr": 5e-5},
    {"params": model.classifier.parameters(), "lr": 1e-4}
])



#optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ======== 6. Función de Entrenamiento ========
def train(model, train_loader, optimizer):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc="Entrenando"):
        # Mover datos al dispositivo
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].float().to(device)  # Convertir etiquetas a float para BCEWithLogitsLoss

        # Reiniciar gradientes
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Loss promedio: {avg_loss}")

for param in model.distilbert.transformer.layer[:4].parameters():
    param.requires_grad = False

# Entrenar por algunas épocas
train(model, train_loader, optimizer)

# Segunda fase: Descongelar 2 capas adicionales
for param in model.distilbert.transformer.layer[2:4].parameters():
    param.requires_grad = True

# Reducir la tasa de aprendizaje y continuar entrenando
for g in optimizer.param_groups:
    g['lr'] = g['lr'] / 2
train(model, train_loader, optimizer)

# ======== 7. Función de Evaluación ========
def evaluate(model, val_loader):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluando"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].float().to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.sigmoid(logits) > 0.5  # Umbral para clasificar como 1 o 0

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Métricas
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels, predictions, average="samples"  # Métricas multietiqueta
    )
    acc = accuracy_score(true_labels, predictions)
    print(f"Accuracy: {acc}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1: {f1}")

# ======== 8. Entrenamiento ========
epochs = 3
for epoch in range(epochs):
    print(f"\n===== Época {epoch + 1} =====")
    train(model, train_loader, optimizer)
    evaluate(model, val_loader)

# ======== 9. Guardar el modelo ========
model.save_pretrained("./fine_tuned_distilbert")
tokenizer.save_pretrained("./fine_tuned_distilbert")



Map:   0%|          | 0/3783 [00:00<?, ? examples/s]

Map:   0%|          | 0/946 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Entrenando: 100%|██████████| 473/473 [00:35<00:00, 13.19it/s]


Loss promedio: 0.5274628179647958


Entrenando: 100%|██████████| 473/473 [00:37<00:00, 12.70it/s]


...

Entrenando: 100%|██████████| 473/473 [00:38<00:00, 12.37it/s]


Loss promedio: 0.2761249307692681


Evaluando: 100%|██████████| 119/119 [00:03<00:00, 33.06it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

Accuracy: 0.7748414376321353
Precision: 0.20718816067653276
Recall: 0.20137420718816068
F1: 0.19978858350951373


('./fine_tuned_distilbert/tokenizer_config.json',
 './fine_tuned_distilbert/special_tokens_map.json',
 './fine_tuned_distilbert/vocab.txt',
 './fine_tuned_distilbert/added_tokens.json')

In [20]:
df2= pd.read_csv('/valuate_dataframe.csv')

In [21]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6047 entries, 0 to 6046
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   clean_tweet  6046 non-null   object
dtypes: object(1)
memory usage: 47.4+ KB


In [63]:
# Filtrar filas donde 'clean_tweet' es nulo
nulos = df2[df2['clean_tweet'].isnull()]
nulos
df2 = df2.dropna(subset=['clean_tweet'])

In [66]:
import pandas as pd
from datasets import Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader
import torch
import numpy as np


tweets = df['clean_tweet'].tolist()


class TweetDataset(Dataset):
    def init(self, tweets):
        self.tweets = tweets

    def len(self):
        return len(self.tweets)

    def getitem(self, idx):
        return self.tweets[idx]

# 1. Cargar el dataset en formato Hugging Face
new_data = Dataset.from_pandas(df2)

# 2. Tokenización
tokenizer = DistilBertTokenizer.from_pretrained("./fine_tuned_distilbert")

def tokenize_function(examples):
    # Asegúrate de que "clean_tweet" sea una lista de cadenas
    return tokenizer(examples["clean_tweet"], padding="max_length", truncation=True, max_length=128)

# Usar map para aplicar la función de tokenización
new_data = new_data.map(tokenize_function, batched=True)

# Establecer el formato para PyTorch
new_data.set_format(type="torch", columns=["input_ids", "attention_mask"])

# 3. Cargar el modelo
model = DistilBertForSequenceClassification.from_pretrained("./fine_tuned_distilbert")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 4. DataLoader
new_loader = DataLoader(new_data, batch_size=8, shuffle=False)

# 5. Hacer predicciones
model.eval()
all_predictions = []
with torch.no_grad():
    for batch in new_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        probs = torch.sigmoid(outputs.logits).cpu().numpy()
        all_predictions.extend(probs)

# 6. Interpretar
columns = ["Vulgar", "Ofensivo"]
predictions_df = pd.DataFrame(all_predictions, columns=columns)
print(predictions_df)


Map:   0%|          | 0/6046 [00:00<?, ? examples/s]

        Vulgar  Ofensivo
0     0.952351  0.818107
1     0.961143  0.794938
2     0.946100  0.916551
3     0.953529  0.377252
4     0.955114  0.657453
...        ...       ...
6041  0.018822  0.134014
6042  0.150121  0.025588
6043  0.032772  0.119256
6044  0.341487  0.057642
6045  0.452894  0.067578

[6046 rows x 2 columns]
